In [1]:
import torch 
from torch import nn 
import torchvision 
from torchvision import transforms

In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
transform = transforms.Compose(
    [transforms.Pad(4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

train_dl = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=8,
                                            prefetch_factor = 2, 
                                            pin_memory = True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

test_dl = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=8,
                                             prefetch_factor = 2, 
                                            pin_memory = True)

val_dl = test_dl

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [4]:
def conv3x3(in_channels, out_channels, stride=1): 
  return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias = False)

class ResidualBlock(nn.Module): 
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
      super(ResidualBlock, self).__init__()
      self.conv1 = conv3x3(in_channels, out_channels, stride)
      self.bn1 = nn.BatchNorm2d(out_channels)
      # self.relu = nn.ReLU(inplace=True)
      self.conv2 = conv3x3(out_channels, out_channels)
      self.bn2 = nn.BatchNorm2d(out_channels)
      self.downsample = downsample

  def forward(self, input): 
        residual = input
        input = nn.ReLU(inplace=True)(self.bn1(self.conv1(input)))
        input = nn.ReLU(inplace=True)(self.bn2(self.conv2(input)))
        if self.downsample: 
          residual = self.downsample(residual)
        input += residual
        return nn.ReLU(inplace=True)(input)

In [28]:
from torch.nn.modules import conv
class ResNet(nn.Module): 
  def __init__(self, block, layers, num_classes=10):
      super(ResNet, self).__init__()
      self.in_channels = 16
      self.conv = conv3x3(3, self.in_channels)
      self.bn = nn.BatchNorm2d(self.in_channels)
      self.relu = nn.ReLU(inplace=True)
      self.layer1 = self.make_layer(block, self.in_channels, layers[0])
      self.layer2 = self.make_layer(block, self.in_channels*2, layers[1], 2)
      self.layer3 = self.make_layer(block, self.in_channels*4, layers[2], 2)
      self.layer4 = self.make_layer(block, self.in_channels*8, layers[3], 2)
      self.avg_pool = nn.AvgPool2d(8)
      self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
      #self.gap = torch.nn.AdaptiveAvgPool2d(1)
      self.fc = nn.Linear(4*self.in_channels, num_classes)

  def make_layer(self, block, out_channels, blocks, stride=1):
      downsample = None
      if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
      layers = []
      layers.append(block(self.in_channels, out_channels, stride, downsample))
      self.in_channels = out_channels
      for i in range(1, blocks):
          layers.append(block(out_channels, out_channels))
      return nn.Sequential(*layers)
  
  def forward(self, input): 
    # Layer 0 
    out = self.conv(input)
    out = self.max_pool(out)
    out = self.bn(out)
    out = nn.ReLU(inplace=True)(out)

    # Layer 1 
    out = self.layer1(out)

    # Layer 2 
    out = self.layer2(out)

    # Layer 3 
    out = self.layer3(out)

    # Layer 4 
    out = self.layer4(out)

    out = self.avg_pool(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out




In [25]:
model = ResNet(ResidualBlock, [2,2,2,2]).to(device)
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [29]:
# https://github.com/JayPatwardhan/ResNet-PyTorch/blob/master/ResNet/ResNet.py

num_epochs = 2

total_step = len(train_dl)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    #if (epoch+1) % 20 == 0:
    #    curr_lr /= 3
    #    update_lr(optimizer, curr_lr)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [14]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dl:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy of the model on the test images: 77.07 %
